In [1]:
# SINGLE-CELL: Fuzzy + TF-IDF matching with robust imports and fallbacks

import pandas as pd
import numpy as np
import re
import nltk
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ---------------- Imports with fallbacks ----------------
FUZZ_LIB = None
try:
    from thefuzz import fuzz, process
    FUZZ_LIB = 'thefuzz'
except Exception:
    try:
        from rapidfuzz import fuzz, process
        FUZZ_LIB = 'rapidfuzz'
    except Exception:
        import difflib
        FUZZ_LIB = 'difflib'

# NLTK stopwords (download if missing)
try:
    from nltk.corpus import stopwords
    _ = stopwords.words('english')
except LookupError:
    nltk.download('stopwords', quiet=True)
    from nltk.corpus import stopwords

from nltk.stem import PorterStemmer

print(f"Using fuzzy lib: {FUZZ_LIB}")

# ---------------- Load data ----------------
resolved_df = pd.read_csv('resolved_queries.csv')
new_df = pd.read_csv('new_queries.csv')

# Normalize expected column names (adjust here if your CSV uses different names)
# Required:
#   resolved_df: 'Query_ID', 'Pre_Resolved_Query'
#   new_df:      'Variation_Query', 'Matches_With_Query_ID'
for col in ['Query_ID', 'Pre_Resolved_Query']:
    assert col in resolved_df.columns, f"Column '{col}' missing in resolved_queries.csv"
for col in ['Variation_Query', 'Matches_With_Query_ID']:
    assert col in new_df.columns, f"Column '{col}' missing in new_queries.csv"

# Basic cleanup
for c in ['Pre_Resolved_Query']:
    resolved_df[c] = resolved_df[c].fillna('')
for c in ['Variation_Query']:
    new_df[c] = new_df[c].fillna('')

print("Resolved queries columns:", resolved_df.columns.tolist())
print("New queries columns:", new_df.columns.tolist())

# ---------------- Preprocessing ----------------
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    tokens = text.split()
    tokens = [stemmer.stem(tok) for tok in tokens if tok not in stop_words]
    return ' '.join(tokens)

resolved_df['processed'] = resolved_df['Pre_Resolved_Query'].apply(preprocess)
new_df['processed'] = new_df['Variation_Query'].apply(preprocess)

# Build IDs & lookups
id_to_processed = dict(zip(resolved_df['Query_ID'], resolved_df['processed']))
id_to_original  = dict(zip(resolved_df['Query_ID'], resolved_df['Pre_Resolved_Query']))

# Note: multiple different Query_IDs can map to the same processed string.
# Keep a list to avoid collisions.
from collections import defaultdict as dd
processed_to_ids = dd(list)
for qid, proc in zip(resolved_df['Query_ID'], resolved_df['processed']):
    processed_to_ids[proc].append(qid)

resolved_processed_list = resolved_df['processed'].tolist()

# ---------------- TF-IDF Vectorizer ----------------
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(resolved_df['processed'])

# ---------------- Fuzzy helpers ----------------
def _extract_one_difflib(query, choices):
    # Returns (match, score) with 0-100 score
    if not choices:
        return None, 0
    best = None
    best_score = -1
    for c in choices:
        score = int(100 * (difflib.SequenceMatcher(None, query, c).ratio()))
        if score > best_score:
            best = c
            best_score = score
    return best, best_score

def get_fuzzy_match(query, choices, threshold=85):
    """
    Try several fuzzy scorers (if available), pick the best.
    Returns (best_match_string, best_score, method_name) or (None, score, method)
    """
    if FUZZ_LIB in ('thefuzz', 'rapidfuzz'):
        methods = [
            ('Simple Ratio', getattr(fuzz, 'ratio')),
            ('Partial Ratio', getattr(fuzz, 'partial_ratio', getattr(fuzz, 'QRatio', getattr(fuzz, 'WRatio', None)))),
            ('Token Sort Ratio', getattr(fuzz, 'token_sort_ratio', getattr(fuzz, 'ratio'))),
            ('Token Set Ratio', getattr(fuzz, 'token_set_ratio', getattr(fuzz, 'ratio'))),
            ('Partial Token Sort Ratio', getattr(fuzz, 'partial_token_sort_ratio', getattr(fuzz, 'ratio')))
        ]
        best_match, best_score, best_method = None, 0, ""
        for method_name, scorer in methods:
            if scorer is None:
                continue
            # both thefuzz and rapidfuzz expose process.extractOne with scorer=
            match = process.extractOne(query, choices, scorer=scorer)
            if match:
                candidate, score = match[0], match[1]
                if score > best_score:
                    best_match, best_score, best_method = candidate, score, method_name
        if best_score >= threshold:
            return best_match, int(best_score), best_method
        return None, int(best_score), best_method
    else:
        # difflib fallback
        match, score = _extract_one_difflib(query, choices)
        method = "difflib SequenceMatcher"
        if score >= threshold:
            return match, int(score), method
        return None, int(score), method

# ---------------- Cosine helper ----------------
def get_cosine_match(query, tfidf_matrix, vectorizer, threshold=0.7):
    qv = vectorizer.transform([query])
    cos = cosine_similarity(qv, tfidf_matrix).flatten()
    idx = int(cos.argmax())
    score = float(cos[idx])
    if score >= threshold:
        return resolved_processed_list[idx], score
    return None, score

# ---------------- Matching loop ----------------
results = []
method_performance = defaultdict(lambda: {'correct': 0, 'total': 0, 'scores': []})

for _, row in new_df.iterrows():
    new_query_proc = row['processed']
    original_query = row['Variation_Query']
    ground_truth_id = row['Matches_With_Query_ID']

    # Get ground-truth processed (may be empty if ID missing)
    gt_processed = id_to_processed.get(ground_truth_id, "")

    # Fuzzy
    fuzzy_match, fuzzy_score, fuzzy_method = get_fuzzy_match(
        new_query_proc, resolved_processed_list, threshold=85
    )
    # Map fuzzy string -> candidate IDs (may be multiple)
    fuzzy_predicted_ids = processed_to_ids.get(fuzzy_match, []) if fuzzy_match else []
    # If multiple, accept correct if any matches GT
    fuzzy_correct = ground_truth_id in fuzzy_predicted_ids if fuzzy_predicted_ids else False
    fuzzy_predicted_id = fuzzy_predicted_ids[0] if fuzzy_predicted_ids else None

    # Cosine
    cosine_match, cosine_score = get_cosine_match(
        new_query_proc, tfidf_matrix, vectorizer, threshold=0.7
    )
    cosine_predicted_ids = processed_to_ids.get(cosine_match, []) if cosine_match else []
    cosine_correct = ground_truth_id in cosine_predicted_ids if cosine_predicted_ids else False
    cosine_predicted_id = cosine_predicted_ids[0] if cosine_predicted_ids else None

    # Track per-method stats
    if fuzzy_method:
        method_performance[fuzzy_method]['correct'] += 1 if fuzzy_correct else 0
        method_performance[fuzzy_method]['total'] += 1
        method_performance[fuzzy_method]['scores'].append(fuzzy_score)

    method_performance['TF-IDF Cosine']['correct'] += 1 if cosine_correct else 0
    method_performance['TF-IDF Cosine']['total'] += 1
    method_performance['TF-IDF Cosine']['scores'].append(cosine_score)

    # Store row
    results.append({
        'Variation_Query': original_query,
        'Ground_Truth_ID': ground_truth_id,
        'Ground_Truth_Query': id_to_original.get(ground_truth_id, ""),
        'Fuzzy_Predicted_ID': fuzzy_predicted_id,
        'Fuzzy_Predicted_Query': id_to_original.get(fuzzy_predicted_id, "") if fuzzy_predicted_id else "",
        'Fuzzy_Method': fuzzy_method if fuzzy_match else "None",
        'Fuzzy_Score': fuzzy_score,
        'Fuzzy_Correct': fuzzy_correct,
        'TFIDF_Predicted_ID': cosine_predicted_id,
        'TFIDF_Predicted_Query': id_to_original.get(cosine_predicted_id, "") if cosine_predicted_id else "",
        'TFIDF_Score': cosine_score,
        'TFIDF_Correct': cosine_correct
    })

results_df = pd.DataFrame(results)

# ---------------- Aggregate metrics ----------------
performance_data = []
for method, stats in method_performance.items():
    total = stats['total']
    if total > 0:
        accuracy = stats['correct'] / total
        avg_score = float(np.mean(stats['scores'])) if stats['scores'] else 0.0
        performance_data.append({
            'Method': method,
            'Accuracy': accuracy,
            'Coverage': total / len(new_df) if len(new_df) else 0.0,
            'Average_Score': avg_score,
            'Correct_Matches': stats['correct'],
            'Total_Matches': total
        })
performance_df = pd.DataFrame(performance_data).sort_values(['Accuracy','Coverage'], ascending=[False, False])

# ---------------- Save & print ----------------
results_df.to_csv('matched_queries_results_individual.csv', index=False)
performance_df.to_csv('method_performance_comparison_individual.csv', index=False)

print(f"\nProcessed {len(new_df)} queries")

print("\nMethod Performance Comparison (Individual Methods Only):")
print(performance_df)

print("\nSample matches:")
cols = ['Variation_Query', 'Ground_Truth_Query',
        'Fuzzy_Predicted_Query', 'Fuzzy_Method', 'Fuzzy_Correct',
        'TFIDF_Predicted_Query', 'TFIDF_Correct']
print(results_df[cols].head(10).to_string(index=False))

# Detailed per-method summary
print("\n\nDetailed analysis by method:")
for method in performance_df['Method'].unique():
    row = performance_df[performance_df['Method'] == method].iloc[0]
    print(f"\n{method}:")
    print(f"  Accuracy: {row['Accuracy']:.2%}")
    print(f"  Coverage: {row['Coverage']:.2%}")
    print(f"  Average Score: {row['Average_Score']:.2f}")
    print(f"  Correct Matches: {int(row['Correct_Matches'])}/{int(row['Total_Matches'])}")

# Best method
if not performance_df.empty:
    best = performance_df.iloc[0]
    print(f"\nBest individual method: {best['Method']} "
          f"with accuracy {best['Accuracy']:.2%} and coverage {best['Coverage']:.2%}")

# Combined stats
fuzzy_methods = [m for m in performance_df['Method'] if m != 'TF-IDF Cosine']
fuzzy_stats = performance_df[performance_df['Method'].isin(fuzzy_methods)]
if not fuzzy_stats.empty:
    total_correct = int(fuzzy_stats['Correct_Matches'].sum())
    total_attempts = int(fuzzy_stats['Total_Matches'].sum())
    overall_acc = total_correct / total_attempts if total_attempts else 0.0
    overall_cov = total_attempts / len(new_df) if len(new_df) else 0.0
    print(f"\nFuzzy Methods Combined:\n  Total Correct: {total_correct}\n  Total Matches: {total_attempts}"
          f"\n  Overall Accuracy: {overall_acc:.2%}\n  Overall Coverage: {overall_cov:.2%}")

tfidf_stats = performance_df[performance_df['Method'] == 'TF-IDF Cosine']
if not tfidf_stats.empty:
    t = tfidf_stats.iloc[0]
    print(f"\nTF-IDF Cosine:\n  Correct Matches: {int(t['Correct_Matches'])}"
          f"\n  Total Matches: {int(t['Total_Matches'])}"
          f"\n  Accuracy: {t['Accuracy']:.2%}\n  Coverage: {t['Coverage']:.2%}")


Using fuzzy lib: difflib


FileNotFoundError: [Errno 2] No such file or directory: 'resolved_queries.csv'